In [1]:
from google.colab import drive

In [ ]:
drive.mount('/content/gdrive')

In [11]:
import pandas as pd
import numpy as np
from scipy.cluster.hierarchy import dendrogram, linkage
import matplotlib.pyplot as plt

In [ ]:
data = pd.read_excel(r'/content/gdrive/My Drive/Colab Notebooks/ЛР3 ПАД_Исходные данные_Бухов.xlsx')
data.head(6)

In [ ]:
# Нормирование данных (Z-нормализация)
mean = np.mean(data, axis=0)
dev = np.std(data, axis=0)
norm_data = (data - mean) / dev

norm_data.head()

In [ ]:
#Дендрограммы для каждого из 3-х типов связей
linkage_methods_arr = ['single', 'complete', 'average'] #ближайший сосед, дальний сосед, расстояние между центрами кластеров
for method in linkage_methods_arr:
  D = linkage(norm_data, method=method)
  plt.figure(figsize=(20, 5))
  dendrogram(D)
  plt.title(f'Иерархическая кластеризация, метод - {method}')
  plt.xlabel('Номер объекта')
  plt.ylabel('Расстояние')
  plt.show()

In [12]:
from sklearn.cluster import AgglomerativeClustering
#Получение итоговых меток кластера для каждого объекта
m = AgglomerativeClustering(n_clusters=4, linkage='complete')
labels = m.fit_predict(norm_data)
print(labels)

[0 0 0 0 0 0 0 0 0 0 3 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 1 0 0 0 1
 0 0 2 0 0 0 0 0 0 0]


In [18]:
from sklearn.metrics import davies_bouldin_score
from sklearn.cluster import KMeans
from sklearn.model_selection import GridSearchCV
#Функция расчета индекса Дэвиса-Болдина
def boldin_scorer(estimator, Data):
  estimator.fit(Data)
  return davies_bouldin_score(Data, estimator.labels_)

#Задаем сетку перебора
kmeans = KMeans(n_init=10)
param_grid = {'n_clusters' : range(2, 9)}
grid_search = GridSearchCV(kmeans, param_grid, scoring = boldin_scorer)

#Перебираем по сетке
grid_search.fit(norm_data)

#Находим оптимальное число кластеров
optimal_clasters_number = grid_search.best_params_['n_clusters']
print('Оптимальное число кластеров: ', optimal_clasters_number)

#Находим kmeans для оптимального числа кластеров
best_kmeans = KMeans(n_init=10, n_clusters=optimal_clasters_number)

#Находим метки
kmeans_labels = best_kmeans.fit_predict(norm_data)
print('Метки кластеров: ', kmeans_labels)



Оптимальное число кластеров:  3
Метки кластеров:  [1 2 2 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 2 1 1 1 1 1 2 0 0 2 2 2 0
 2 2 2 2 2 2 2 2 2 1]


In [19]:
from sklearn.metrics import rand_score
#Расчитываем индекс Рэнда двух полученных моделей
rand_index = rand_score(kmeans_labels, labels) #labels - метки для дальнего соседа
print(rand_index)

0.5920444033302498


In [20]:
clusters_data = norm_data.copy()
clusters_data['Cluster'] = kmeans_labels
cluster_means = clusters_data.groupby('Cluster').mean()
print(cluster_means)

               х1        х2        х3        х4        х5        х6        x7  \
Cluster                                                                         
0        2.251718 -0.705718 -0.983481 -1.492615  0.308362 -1.159806 -1.218018   
1       -0.489067  0.615531  0.294335  0.015934 -0.400075  0.182768 -0.210947   
2        0.218170 -0.775349 -0.218752  0.326834  0.539324 -0.006632  0.609217   

               x8        x9  
Cluster                      
0        2.887118  0.697045  
1       -0.277574 -0.453689  
2       -0.254797  0.529867  
